# 0.1 라이브러리 import

In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dset
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torch.autograd import Variable
import numpy as np

# 0.2 하이퍼파라미터 셋팅

In [2]:
batch_size=128
num_epochs=10

word_vec_size=256
dropout_p=0.3

hidden_size=512
num_layers=4

learning_rate=0.001

In [3]:
# Device configuration
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 1. SMS train, test dataset 가져오기

In [4]:
from data_loader import DataLoader

In [5]:
loaders=DataLoader(
    train_fn='./sms.maxlen.uniq.shuf.train.tsv',
    batch_size=batch_size,
    valid_ratio=.2, #8:2
    device=-1,
    max_vocab=999999,
    min_freq=5,
)

In [6]:
test_loaders=DataLoader(
    train_fn='./sms.maxlen.uniq.shuf.test.tsv',
    batch_size=batch_size,
    valid_ratio=.01, #모두 trai
    device=-1,
    max_vocab=999999,
    min_freq=5,
)

# 2. 대략적인 데이터 형태

In [7]:
print("|train| = ",len(loaders.train_loader.dataset),
      "|valid| =", len(loaders.valid_loader.dataset))
vocab_size=len(loaders.text.vocab)
num_classes=len(loaders.label.vocab)
print('|vocab| = ',vocab_size, '|classes| =',num_classes)


|train| =  3722 |valid| = 930
|vocab| =  1549 |classes| = 2


# 3. 데이터 로드함수
- 학습시킬 때 batch_size 단위로 끊어서 로드하기 위함!

In [8]:
n=3 #샘플로 출력할 데이터의 개수
for i, data in enumerate(loaders.train_loader):
    labels=data.label
    texts=data.text
    
    if i>n:
        break
    print("[%d]" %i)
    print("한 번에 로드되는 데이터 크기:", len(labels))
    
    #출력
    for j in range(n):
        label=labels[j].numpy() #tensor를 numpy로 변환
        text=texts[j].numpy()
        print("label: ",label)
        print("text: ",text.shape)

[0]
한 번에 로드되는 데이터 크기: 128
label:  0
text:  (24,)
label:  0
text:  (24,)
label:  0
text:  (24,)
[1]
한 번에 로드되는 데이터 크기: 128
label:  0
text:  (4,)
label:  0
text:  (4,)
label:  0
text:  (4,)
[2]
한 번에 로드되는 데이터 크기: 128
label:  0
text:  (7,)
label:  0
text:  (7,)
label:  0
text:  (7,)
[3]
한 번에 로드되는 데이터 크기: 128
label:  1
text:  (22,)
label:  1
text:  (22,)
label:  1
text:  (22,)


# 4. 모델 선언

In [9]:
class RNN(nn.Module):
    def __init__(self,
                 input_size,
                 word_vec_size,
                 hidden_size, 
                 n_classes,
                 num_layers=4,
                 dropout_p=0.3
                ):
        super(RNN, self).__init__()
        
        self.input_size=input_size
        self.sord_vec_size=word_vec_size
        self.hidden_size = hidden_size
        self.n_classes=n_classes
        self.num_layers = num_layers
        self.dropout_p=dropout_p
        
        #입력차원(vocab_size), 출력차원(word_vec_size)
        self.emb=nn.Embedding(input_size,word_vec_size)
        
        self.lstm=nn.LSTM(input_size=word_vec_size,
                         hidden_size=hidden_size,
                         num_layers=num_layers,
                             dropout=dropout_p,
                             batch_first=True,
                             bidirectional=True)
        
        self.fc = nn.Linear(hidden_size*2, num_classes)
        self.activation=nn.LogSoftmax(dim=1)
        
    def forward(self, x):
        x=self.emb(x)
        
        x,_=self.lstm(x)
        
        out=self.activation(self.fc(x[:,-1]))
        
        return out

In [10]:
model=RNN(input_size=vocab_size,
         word_vec_size=word_vec_size,
         hidden_size=hidden_size,
         n_classes=num_classes,
         num_layers=num_layers,
         dropout_p=dropout_p)

In [11]:
def ComputeAccr(dloader, imodel):
    correct=0
    total=0
    
    model.eval() #test mode
    for i,data in enumerate(dloader): #batch_size 만큼
        texts=data.text.to(device)
        labels=data.label.to(device)
        
        # Forward prop.
        output=model(texts) #(batch_size, num_classes)
        _, output_index=torch.max(output,1) # (batch_size, 1)
        
        total+=labels.size(0)
        correct+=(output_index==labels).sum().float()
    model.train()
    return (100*correct/total).cpu().numpy() # tensor -> numpy 

In [12]:
print("Accuracy of Test Data: %.2f" %ComputeAccr(loaders.valid_loader, model))

Accuracy of Test Data: 10.97


# 5. loss, optimizer

In [13]:
loss_func=nn.NLLLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)

# 6. 학습

In [14]:
# Train the model
total_step = len(loaders.train_loader)
for epoch in range(num_epochs):
    for i, data in enumerate(loaders.train_loader):
        texts=data.text.to(device) #(batch_size,length)
        labels=data.label.to(device) #batch_size.num_classes
        
        print("[%d]" %i)
        
        # Forward pass
        outputs = model(texts)
        loss = loss_func(outputs, labels)
        
        # Backward prop.& optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 10 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accr:{:.2f}' 
                   .format(epoch+1, num_epochs, i+1, total_step,
                           loss.item(),
                           ComputeAccr(loaders.valid_loader,model)))

[0]
[1]
[2]
[3]
[4]
[5]
[6]
[7]
[8]
[9]
Epoch [1/10], Step [10/30], Loss: 0.7219, Accr:12.15
[10]
[11]
[12]
[13]
[14]
[15]
[16]
[17]
[18]
[19]
Epoch [1/10], Step [20/30], Loss: 0.3237, Accr:89.03
[20]
[21]
[22]
[23]
[24]
[25]
[26]
[27]
[28]
[29]
Epoch [1/10], Step [30/30], Loss: 0.1236, Accr:89.03
[0]
[1]
[2]
[3]
[4]
[5]
[6]
[7]
[8]
[9]
Epoch [2/10], Step [10/30], Loss: 0.6869, Accr:89.03
[10]
[11]
[12]
[13]
[14]
[15]
[16]
[17]
[18]
[19]
Epoch [2/10], Step [20/30], Loss: 0.1732, Accr:89.03
[20]
[21]
[22]
[23]
[24]
[25]
[26]
[27]
[28]
[29]
Epoch [2/10], Step [30/30], Loss: 0.9942, Accr:89.03
[0]
[1]
[2]
[3]
[4]
[5]
[6]
[7]
[8]
[9]
Epoch [3/10], Step [10/30], Loss: 0.1750, Accr:89.03
[10]
[11]
[12]
[13]
[14]
[15]
[16]
[17]
[18]
[19]
Epoch [3/10], Step [20/30], Loss: 0.6406, Accr:89.03
[20]
[21]
[22]
[23]
[24]
[25]
[26]
[27]
[28]
[29]
Epoch [3/10], Step [30/30], Loss: 1.1027, Accr:89.03
[0]
[1]
[2]
[3]
[4]
[5]
[6]
[7]
[8]
[9]
Epoch [4/10], Step [10/30], Loss: 0.8481, Accr:89.03
[10]
[11]


# 7. 테스트

In [15]:
print("Accuracy of valid Data: %.2f" %ComputeAccr(loaders.valid_loader, model))

Accuracy of valid Data: 89.03


# 8. 학습된 파라미터 저장

In [ ]:
netname='./nets/rnn_weight.pkl'
torch.save(model, netname,)

# 9. 학습된 파라미터 로드

In [ ]:
netname='./nets/rnn_weight.pkl'
model=torch.load(netname)

In [ ]:
print("Accuracy of Test Data: %.2f" %ComputeAccr(test_loader, model))